<a href="https://colab.research.google.com/github/ahmadhajmosa/3d-force-graph/blob/master/Gradient_activations_funcs_comparision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download MNIST dataset

In [0]:
from keras.datasets import mnist
import pandas as pd

import keras
import keras.backend as K

from keras.layers import Input, Dense, BatchNormalization, add
from keras.models import Sequential, Model
from keras.initializers import random_uniform, glorot_uniform, he_normal,he_uniform,glorot_uniform,glorot_normal,lecun_normal,Orthogonal
import tensorflow as tf
import numpy as np


tf.compat.v1.random.set_random_seed(1234) #fix seeds
# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()


#
num_classes = 10
epochs = 12

x_train = x_train.reshape(x_train.shape[0], img_rows * img_cols)
x_test = x_test.reshape(x_test.shape[0], img_rows * img_cols)

x_train= x_train/255 # standardization of the data


input_shape = img_rows * img_cols


y_train = pd.get_dummies(y_train).values

# Model Creation 

In [10]:


# get gradient norm of each layer

def get_gradient_norm(model,layer_index):
        with K.name_scope('gradient_norm'):
            grads = K.gradients(model.total_loss, model.layers[layer_index].input)
            norm = K.sqrt(sum([K.sum(K.square(g)) for g in grads]))        
        return norm
      
# get abs grad of all neurons
   
def get_abs_gradients(model,layer_index):
        with K.name_scope('gradient_norm'):
            grads = K.abs(K.gradients(model.total_loss, model.layers[layer_index].input))
        return grads     

 
def test_model(num_h_layers,activation,kernel_init, with_skip = False):

      
    activations = []  # neurons activations list
    kernel_init = eval(kernel_init+ '(seed={})'.format(1)) #
    
    inp = Input(shape=(input_shape,))
    
    z = Dense(100,activation=activation,kernel_initializer=kernel_init,use_bias=False)(inp)  # first hidden layer
    
    for i in range(num_h_layers): # rest of hidden layers
      
        x = Dense(100,activation=activation,kernel_initializer=kernel_init,use_bias=False)(z)
        
        if with_skip: # with skip connections
          z = add([z,x])
        else:
          z=x
        activations.append(z)
        
    
    # output layer

    x = Dense(num_classes,activation='softmax',kernel_initializer=kernel_init,use_bias=False)(z)  

    model = Model(inp,x)
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])

    
    # creating the gradient checking list of tensor
    gradient = []
    for la_ind in range(0,num_h_layers-2,1):
      gradient.append(get_gradient_norm(model,la_ind+1))
      
      
    gradient_tensors = K.stack(gradient)  # stacking the gradient tensors
    activation_tensors = K.stack(activations)  # stacking the activation tensors

    # adding the gradient tensors to the metrics
    model.metrics_names.append("weights_gradient")
    
    model.metrics_tensors.append(gradient_tensors)

    # adding the activation tensors to the metrics
    model.metrics_names.append("activations")
   
    model.metrics_tensors.append(activation_tensors)    

    # train for 5 iterations using a batch of 10 images
    gradients=[]
    accs = []
    acts=[]
    for batch in range(5):

      _,acc,grad, act = model.train_on_batch(x_train[:10],y_train[:10])  # batch with th size 1
      gradients.append(grad)
      acts.append(act)
      accs.append(acc)
      
    return gradients, acts, accs


# call the model using 15 layers with different activation functions
gradients = dict()
activations = dict()
accs=dict()
N =15
activations_list=['relu','elu','softplus','tanh','sigmoid']
kernel_init_list=['he_normal','he_uniform','glorot_uniform','glorot_normal','lecun_normal','Orthogonal']

for activation in activations_list:
  for kernel_init in kernel_init_list:
      gradients[activation + ' + ' +kernel_init], activations[activation + ' + ' + kernel_init],accs[activation + ' + ' + kernel_init]=test_model(N,activation,kernel_init)


W0629 14:11:28.389805 139846961162112 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



# Plotly Layout Configration 

In [0]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
#plotly.tools.set_credentials_file(username='', api_key='')

layout = go.Layout(
    xaxis=dict(
                title='Depth of the network: 0 is the input layer',

               titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            color='black'
        ),
        tickmode='linear',
        ticks='outside',
        tick0=0,
        dtick=1,
        ticklen=8,
        tickwidth=4,
        tickcolor='#000'
    ),
    yaxis=dict(
         title='Training Epochs',

               titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            color='black'
        ),
        tickmode='linear',
        ticks='outside',
        tick0=0,
        dtick=1,
        ticklen=8,
        tickwidth=4,
        tickcolor='#000'
    )
)

# Reporting

#ELU

In [24]:




display(accs['elu + glorot_uniform'])
trace = go.Heatmap(z=gradients['elu + glorot_uniform'])
data=[trace]


fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='elu')




[0.0, 0.6, 0.8, 0.8, 0.8]

#Relu

In [25]:
import plotly.plotly as py
import plotly.graph_objs as go



display(accs['relu + glorot_uniform'])
trace = go.Heatmap(z=gradients['relu + glorot_uniform'])
data=[trace]


fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='relu')




[0.1, 0.1, 0.1, 0.1, 0.1]

# Softplus

In [30]:

display(accs['softplus + glorot_uniform'])
trace = go.Heatmap(z=gradients['softplus + glorot_uniform'])
data=[trace]


fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='softplus')

[0.1, 0.1, 0.3, 0.3, 0.3]

# Sigmoid

In [31]:

display(accs['sigmoid + glorot_uniform'])
trace = go.Heatmap(z=gradients['sigmoid + glorot_uniform'])
data=[trace]


fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='sigmoid')

[0.1, 0.1, 0.1, 0.2, 0.3]

# Tanh

In [32]:

display(accs['tanh + glorot_uniform'])
trace = go.Heatmap(z=gradients['tanh + glorot_uniform'])
data=[trace]


fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='tanh')

[0.0, 0.8, 0.9, 0.9, 0.9]